In [1]:
import json
from src.utils import logging, logger, send_log
import backtrader as bt
import btoandav20 as bto

ACCOUNT_CONFIG_FILE = 'src/config/oanda.json'
OANDA_IS_LIVE = False
INSTRUMENT = 'EUR_USD'

# Load configurations
with open(ACCOUNT_CONFIG_FILE) as f:
    d_accounts = json.load(f)

# Oanda API configuration
oanda_token = d_accounts['oanda_token']
oanda_account_id = d_accounts['oanda_account_id']

# Config logger
logger.setLevel(logging.DEBUG)


In [2]:
# Oanda store configuration
config_oanda_store = dict(
    token=oanda_token,
    account=oanda_account_id,
    practice=not OANDA_IS_LIVE,
    notif_transactions=True,
    stream_timeout=10,
)
oanda_store = bto.stores.OandaV20Store(**config_oanda_store)

# Oanda data configuration
config_oanda_data = dict(
    timeframe=bt.TimeFrame.Minutes,
    compression=1,
    tz='America/New_York',
    backfill=False,
    backfill_start=False,
)
oanda_data = oanda_store.getdata(dataname=INSTRUMENT, **config_oanda_data)
oanda_data.resample(
    timeframe=bt.TimeFrame.Minutes,
    compression=1,
)

In [3]:
# Define trading strategy
ORDER_SIZE = 10
SL_DISTANCE = 0.00025
TP_DISTANCE = 0.00025

class TestStrategy(bt.Strategy):

    def __init__(self):
        # Display data parameters
        data_params = self.datas[0].params
        send_log(f"INITIALIZE STRATEGY: instrument = {data_params.dataname}, timezone = {data_params.tz}, timeframe = {data_params.timeframe}")

        self.dataclose = self.datas[0].close

        self.order = None
        
    def next(self):
        self._log(f"close = {self.dataclose[0]:.5f}, previous close = {self.dataclose[-1]:.5f}", level=logging.DEBUG)

        # Avoid duplicated order if there is already a pending order
        if self.order:
            return

        if not self.position:
            current_close = self.dataclose[0]
            if current_close > self.dataclose[-1]:
                sl_price = current_close - SL_DISTANCE
                tp_price = current_close + TP_DISTANCE
                self._log(f"CREATE ORDER - MARKET BUY: sl = {sl_price:.5f}, tp = {tp_price:.5f}")
                self.order = self.buy_bracket(
                    size=ORDER_SIZE,
                    exectype=bt.Order.Market,
                    stopprice=sl_price,
                    stopexec=bt.Order.Stop,
                    limitprice=tp_price,
                    limitexec=bt.Order.Limit,
                )
            if current_close < self.dataclose[-1]:
                sl_price = current_close + SL_DISTANCE
                tp_price = current_close - TP_DISTANCE
                self._log(f"CREATE ORDER - MARKET SELL: sl = {sl_price:.5f}, tp = {tp_price:.5f}")
                self.order = self.sell_bracket(
                    size=ORDER_SIZE,
                    
                    exectype=bt.Order.Market,
                    stopprice=sl_price,
                    stopexec=bt.Order.Stop,
                    limitprice=tp_price,
                    limitexec=bt.Order.Limit,
                )

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            self._log_order(order)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self._log(f"ORDER CANCELED / MARGIN / REJECTED: {order.status}")
        
        else:
            self._log(f"WARNING: unknown order status: {order.status}", level=logging.WARNING)

        # Write down - no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self._log_trade(trade)

    def _log(self, txt, level=logging.INFO):
        dt = self.datas[0].datetime.datetime(0).strftime("%Y-%m-%d %H:%M:%S")
        send_log(f"({dt}) {txt}", level=level)
    
    def _log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        s_comm = f", comm = {order.executed.comm:.5f}" if order.executed.comm > 0 else ""
        self._log(f"ORDER EXECUTED - {direction}: size = {order.executed.size}, price = {order.executed.price:.5f}, cost = {order.executed.value:5f}{s_comm}")

    def _log_trade(self, trade: bt.trade.Trade):
        self._log(f"TRADE COMPLETED: value = {trade.value:.5f}, gross profit = {trade.pnl:.5f}, net profit = {trade.pnlcomm:.5f}")


In [4]:
# Initialize backtrader
cerebro = bt.Cerebro()

cerebro.adddata(oanda_data)
cerebro.setbroker(oanda_store.getbroker())
cerebro.addstrategy(TestStrategy)


0

In [5]:
cerebro.run()

[utils.py:30 - send_log()] INITIALIZE STRATEGY: instrument = EUR_USD, timezone = America/New_York, timeframe = 4
[utils.py:30 - send_log()] (2023-03-21 05:15:00) close = 1.07328, previous close = 1.07328
[utils.py:30 - send_log()] (2023-03-21 05:16:00) close = 1.07329, previous close = 1.07328
[utils.py:30 - send_log()] (2023-03-21 05:16:00) CREATE ORDER - MARKET BUY: sl = 1.07304, tp = 1.0735400000000002
[utils.py:30 - send_log()] (2023-03-21 05:17:00) ORDER EXECUTED - BUY: size = 10.0, price = 1.07338, cost = 0.000000
[utils.py:30 - send_log()] (2023-03-21 05:17:00) close = 1.07344, previous close = 1.07329
[utils.py:30 - send_log()] (2023-03-21 05:18:00) close = 1.07335, previous close = 1.07344
[utils.py:30 - send_log()] (2023-03-21 05:19:00) close = 1.07330, previous close = 1.07335
[utils.py:30 - send_log()] (2023-03-21 05:20:00) close = 1.07329, previous close = 1.07330
[utils.py:30 - send_log()] (2023-03-21 05:21:00) close = 1.07327, previous close = 1.07329
[utils.py:30 - send

KeyboardInterrupt: 

In [ ]:
# Define trading strategy
class TestStrategy(bt.Strategy):

    def __init__(self):
        # Display data parameters
        data_params = self.datas[0].params
        logger.info(f"INITIALIZE STRATEGY: instrument = {data_params.dataname}, timezone = {data_params.tz}, timeframe = {data_params.timeframe}")

        self.dataclose = self.datas[0].close

        self.order = None
        
    def next(self):
        self._log(f"close price = {self.dataclose[0]:.5f}, previous close price = {self.dataclose[-1]:.5f}", level=logging.DEBUG)

        # Avoid duplicated order if there is already a pending order
        if self.order:
            return

        if not self.position:
            if self.dataclose[0] > self.dataclose[-1]:
                self._log(f"CREATE ORDER - MARKET BUY")
                self.order = self.buy()
            if self.dataclose[0] < self.dataclose[-1]:
                self._log(f"CREATE ORDER - MARKET SELL")
                self.order = self.sell()
        else:
            position = self.getposition()
            if position.size > 0:
                if self.dataclose[0] < self.dataclose[-1]:
                    self._log(f"CREATE ORDER - MARKET SELL: to close long position")
                    self.order = self.sell()
            elif position.size < 0:
                if self.dataclose[0] > self.dataclose[-1]:
                    self._log(f"CREATE ORDER - MARKET BUY: to close short position")
                    self.order = self.buy()
            else:
                self._log(f"WARNING: position without size", level=logging.WARNING)

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            self._log_order(order)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self._log(f"ORDER CANCELED / MARGIN / REJECTED: {order.status}")
        
        else:
            self._log(f"WARNING: unknown order status: {order.status}", level=logging.WARNING)

        # Write down - no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self._log_trade(trade)

    def _log(self, txt, level=logging.INFO):
        dt = self.datas[0].datetime.datetime(0).strftime("%Y-%m-%d %H:%M:%S")
        logger.log(msg=f"({dt}) {txt}", level=level)
    
    def _log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        s_comm = f", comm = {order.executed.comm:.5f}" if order.executed.comm > 0 else ""
        self._log(f"ORDER EXECUTED - {direction}: size = {order.executed.size}, price = {order.executed.price:.5f}, cost = {order.executed.value:5f}{s_comm}")

    def _log_trade(self, trade: bt.trade.Trade):
        self._log(f"OPERATION PROFIT: gross = {trade.pnl:.5f}, net = {trade.pnlcomm:.5f}")


In [ ]:
# Define trading strategy
class SingleMovingAvgStrategy(bt.Strategy):
    params = (
        ('sma_period', 5),
    )

    def __init__(self):
        logger.info("Initializing Strategy")
        self.dataclose = self.datas[0].close
        self.order = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0],
            period=self.params.sma_period
        )
        
    def next(self):
        self._log(f"Close Price: {self.dataclose[0]:.5f}")

        # Avoid duplicated order if there is already a pending order
        if self.order:
            return

        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self._log(f"BUY MARKET ORDER CREATE")
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                self._log(f"SELL MARKET ORDER CREATE")
                self.order = self.sell()

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            self._log_order(order)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self._log(f"Order CANCELED / MARGIN / REJECTED: {order.status}")
        
        else:
            self._log(f"Warning! Unknown order status: {order.status}")

        # Write down - no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self._log(f"OPERATION PROFIT, Gross {trade.pnl:.5f}, Net {trade.pnlcomm:.5f}")

    def _log(self, txt):
        dt = self.datas[0].datetime.date(0)
        logger.info(f"[{dt.isoformat()}] {txt}")
    
    def _log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        self._log(f"{direction} EXECUTED, Size: {order.executed.size}, Price: {order.executed.price:.5f}, Cost: {order.executed.value:5f}, Comm: {order.executed.comm:.5f}")
